# T3: Matrices

```{note}
Click the {fa}`rocket` and open this notebook in Colab to enable interactivity.
```

```{note}
To save your progress, make a copy of this notebook in Colab `File > Save a copy in Drive` and you'll find it in `My Drive > Colab Notebooks`.
```

Now that we have experience with vectors (1D NumPy arrays), it's time to move onto matrices, or **2D NumPy arrays**!
Matrices are the workhorse of linear algebra and are foundational to physics-based models, computer graphics, machine learning, and so much more!

## Introduction

A matrix can be defined in Python as row vectors stacked on top of each other. 
As such, we can use the same [`np.array(obj)`](https://numpy.org/doc/stable/reference/generated/numpy.array.html) constructor where `obj` is now a **list of lists**, one for each row of the matrix.



In [ ]:
import numpy as np
A = np.array([ [1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12] ])
print(A)
print(A.shape)

```{note}
We will also try to be consistent with our variable names: lowercase for scalars/vectors, uppercase for matrices.
```

All rows must have the same length or NumPy will raise an error.

In [ ]:
np.array([ [1, 2, 3], [4, 5, 6, 7] ])

If a matrix is assigned to a variable, its entries can be referred to by a 2-number subscript in square brackets `[_, _]`.
The first number in the subscript is the row index. 
The second number in the subscript is the column index.
Remember that Python is indexed starting from `0`!

In [ ]:
A = np.array([ [1, 2, 3], [4, 5, 6] ])
print(A)
print(A[1, 1])
A[0, 2] = -2
print(A)
A[2, 0] = 7

```{note}
Note that unlike MATLAB, NumPy arrays cannot be _dynamically resized_.
That is, you cannot reference indices outside of the range of the allocated matrix.
```

To compute the transpose of a matrix, use the `A.T` operator on matrix `A`.
To compute the conjugate transpose of a matrix, use the `A.conj().T`.
When you matrix doesn't contain imaginary parts, the two types of transpose give the same result (fortunately, this is always the case in this class).

In [ ]:
print(A.T)
print(A.conj().T)

A row vector can be thought of as a matrix with 1 row, though in NumPy there is some nuance.
A column vector is a matrix with 1 column (again, some nuance in NumPy).
A column vector can be made by transposing a row vector, generally speaking, although it doesn't always work.
The following _does work_, but what happens if you remove the `ndmin` parameter?

In [ ]:
x = np.array([1, 2, 3, 4, 5], ndmin=2)
print(x)
print(x.T)

To create a zero matrix of a specified dimension (`shape`), use the [`np.zeros(shape)`](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html) command.

In [ ]:
print(np.zeros([3, 5]))
print(np.zeros(4))

Analogously, to create a matrix whose entries are all `1`, use the [`np.ones(shape)`](https://numpy.org/doc/stable/reference/generated/numpy.ones.html) command.

In [ ]:
print(np.ones([3, 5]))
print(np.ones(4))

To create an identity matrix, use the [`np.eye(M, N)`](https://numpy.org/doc/stable/reference/generated/numpy.eye.html) command.

In [ ]:
print(np.eye(3, 5))
print(np.eye(4))

To obtain the dimensions of the matrix, use the `.shape` attribute.
The `.size` attributes returns the total number of elements.

```{warning}
These are the opposite keywords of MATLAB!
```

In [ ]:
print(A.shape)
print(A.shape[0])
print(A.shape[1])
print(A.size)

Element-wise operations work on matrices of the same dimension.
If one of the two operands is a scalar, **scalar expansion** will take place.
Unary scalar functions can be applied to matrices, and they will operate element-wise.

In [ ]:
2 * np.eye(3) - np.ones(3)

In [ ]:
A = np.array([ [np.pi/6, np.pi/4], [np.pi/3, np.pi/2] ])
np.sin(A)

In [ ]:
np.log(np.arange(1, 11)) > 2

## Concatenation

Matrices that have consistent dimensions can be concatenated, either vertically or horizontally, using the [`np.concatenate(arr_list, axis)`](https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html) function.
The first argument is a list of arrays, and the `axis` parameter specifies vertical (`axis=0`) or horizontal (`axis=1`) concatenation.

In [ ]:
# vertical
A = np.array([ [1, 2], [3, 4] ])
B = np.array([[5, 6]])
print(A, B)
print(np.concatenate([A, B], axis=0))

# horizontal
print(np.concatenate([A, B.T], axis=1))

Obviously, matrices with mismatched dimensions cannot be concatenated.

In [ ]:
np.concatenate([np.ones((3,2)), np.ones((2,2))], axis=1)

Nested concatenations are possible, if you so desire.

In [ ]:
A = np.eye(3)
B = np.ones((3,2))
C = np.zeros((4,5))
np.concatenate([np.concatenate([A, B], axis=1), C], axis=0)

## Multiplication and division

The `@` operator in Python is **matrix-matrix multiplication**.
The product `A @ B` will be defined only if `A.shape[1]` and `B.shape[0]` are equal.

In [ ]:
A = np.array([[1, 2], [3, 4]])
B = np.array([[1, 2, 3], [4, 5, 6]])
print(A @ B)
print(B @ A)

On the other hand, the `*` operator is **element-wise multiplication**, or scalar-matrix multiplication.

In [ ]:
print(2 * A)
print(A * A)

Matrices can also be divided element-wise by a scalar, or another matrix of the same dimensions, with the `/` operator.

In [ ]:
print(A / 2)
print(A / A)

Exponentiation can only be done on a square matrix using [`np.linalg.matrix_power(A, n)`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.matrix_power.html), and the exponent `n` has to be an integer.

In [ ]:
print(np.linalg.matrix_power(A, 3))
print(A @ A @ A)

## System of linear equations

### Example 

A system of linear equations can be solved by putting the augmented matrix into its reduced row echelon form.
We will demonstrate this by solving the network flow problem in the picture below for the four unknown flows.

![network_flow](../assets/fig/network_flow.png)

#### Step 1

Identify the system of equations to solve by matching inflow and outflow at each of the nodes:

$$ \begin{bmatrix} 1 & 1 & 0 & 0 \\ 1 & 0 & -1 & 0 \\ 0 & 1 & 0 & -1 \\ 0 & 0 & 1 & 1 \end{bmatrix} \begin{bmatrix} w \\ x \\ y \\ z \end{bmatrix} = \begin{bmatrix} 5 \\ 2 \\ 3 \\ 0 \end{bmatrix} $$

#### Step 2

Construct the augmented matrix and reduce. 
After the sequence of operations, you should get

$$ \left(\begin{array}{rrrr|r}  
1 & 0 & 0 & 1 & 2 \\
0 & 1 & 0 & -1 & 3 \\
0 & 0 & 1 & 1 & 0 \\
0 & 0 & 0 & 0 & 0
\end{array}\right) $$

#### Step 3

Read off the solution(s), if any.

Here, there are infinitely many solutions, as there's one degree of freedom.
If we pick $z$ as the free parameter, other variables are defined by $w = 2-z$, $x = 3+z$, and $y=-z$.

#### Python verification

We can produce the reduced row echelon form of a matrix using the [`rref()`](https://docs.sympy.org/latest/tutorials/intro-tutorial/matrices.html#rref) method from the [SymPy](https://www.sympy.org/en/index.html) library.
Unfortunately there's no built-in method in NumPy, but the SymPy library has its own advantages (look at that $\LaTeX$-ified output!).

```{note}
Depending on your Python environment, you may have to install SymPy before you can import it, i.e., `pip install sympy`. 
Luckily, Colab comes with it pre-installed!
```

In [ ]:
# Python verification
import numpy as np
from sympy import Matrix

A = np.array([[1, 1, 0, 0], [1, 0, -1, 0], [0, 1, 0, -1], [0, 0, 1, 1]])
b = np.array([[5], [2], [3], [0]])

aug = Matrix(np.concatenate([A, b], axis=1))
display(aug.rref()[0])

print(A @ np.array([[2], [3], [0], [0]]) - b)
print(A @ np.array([[1], [4], [-1], [1]]) - b)

### Your turn

Repeat the previous problem, but with number $3$ (left flow on the top) replaced by $4$.

#### TODO: Write your solution below

What is augmented matrix and reduced augmented matrix?



## Range indexing (optional)

You can take certain elements of a vector and put them together into a new vector by supplying a vector of indices as the subscript. 

In [ ]:
u = np.arange(100, 54, -5)
print(u[[1, 4, 2, -1, 5]])
print(u[np.arange(3, 10, 2)])
print(u[np.arange(4, -1, -1)])

This works with matrices too, as we'll show below with a fancy [Kronecker product](https://numpy.org/doc/stable/reference/generated/numpy.kron.html) and a [Vandermonde matrix](https://numpy.org/doc/stable/reference/generated/numpy.vander.html).

In [ ]:
A = np.kron(np.array([[1], [2]]), np.vander(np.arange(1, 6)))
A[[1, 3, -1], 1:4]

To refer to a whole column or a whole row, use `:` as the corresponding index.

In [ ]:
print(A[-1, :])
print(A[:, [3, 1]])

Assignment to a block in a matrix can be done if the right-hand side is a matrix of the same dimension.

In [ ]:
A[1:5, 2:4] = np.eye(4, 2)
print(A)
A[1:5, 2:4] = np.ones((3, 3))

The right-hand side of a block assignment can also be a scalar, in which case all entries in the block will be set to the same scalar. 
This is another instance of scalar expansion.

In [ ]:
A = np.kron(np.array([[1], [2]]), np.vander(np.arange(1, 6)))
A[[2, 4, 6], [0, 2, 4]] = 9
print(A)
A[:, :] = 0
print(A)
A = 0
print(A)   # note the difference!

## Elementary row operations (optional)

SymPy can put a matrix into the reduced row echelon form for you, but you may not see each step
of the reduction. 
If you want to perform row operations manually, you can do so using range indexing as follows.

In [ ]:
import numpy as np
A = np.array([[1, 2, 3], [2, 5, 8], [3, 4, 5]])
print(A)

# subtract 2 * Row1 from Row2
A[1, :] = A[1, :] - 2 * A[0, :]

# subtract 3 * Row1 from Row3
A[2, :] = A[2, :] - 3 * A[0, :]

# TODO: Try more on your own and compare with the sympy solution
